In [1]:
import torch
import torch.nn as nn

In [2]:
from data_preparation import load_data_tensor

lr_train, lr_test, hr_train = load_data_tensor("dgl-icl")


In [3]:
A = torch.randint(0, 3, (2, 2, 2))
B = torch.randint(0, 3, (2, 2, 2))

In [4]:
A[1] @ B[1]

tensor([[2, 4],
        [5, 2]])

In [5]:
B

tensor([[[1, 1],
         [0, 0]],

        [[1, 2],
         [2, 0]]])

In [6]:
torch.bmm(A, B)

tensor([[[0, 0],
         [1, 1]],

        [[2, 4],
         [5, 2]]])

In [7]:
lr_train.shape
lr_stack = lr_train.reshape(-1, 160)
hr_stack = hr_train.reshape(-1,268)
print(lr_stack[0])

tensor([0.0000, 0.3388, 0.2025, 0.6895, 0.6105, 0.4464, 0.6495, 0.6894, 0.4389,
        0.5773, 0.0000, 0.4907, 0.2030, 0.5973, 0.1080, 0.5209, 0.2436, 0.3475,
        0.3142, 0.4680, 0.4328, 0.6836, 0.1751, 0.4156, 0.2886, 0.4879, 0.5107,
        0.0409, 0.6092, 0.3186, 0.3773, 0.5973, 0.4236, 0.5157, 0.0000, 0.2851,
        0.0880, 0.1500, 0.0387, 0.1536, 0.3912, 0.6067, 0.0019, 0.2677, 0.3644,
        0.1659, 0.2804, 0.0000, 0.2246, 0.0000, 0.0168, 0.0154, 0.2328, 0.1669,
        0.3741, 0.0000, 0.0000, 0.3887, 0.5526, 0.0068, 0.5261, 0.0912, 0.3366,
        0.1353, 0.0618, 0.1170, 0.3364, 0.1876, 0.2191, 0.0557, 0.0009, 0.1510,
        0.0694, 0.0450, 0.2801, 0.0509, 0.0000, 0.2265, 0.2669, 0.0807, 0.7052,
        0.1387, 0.1906, 0.3906, 0.0000, 0.2254, 0.4377, 0.2353, 0.0193, 0.2244,
        0.0000, 0.2057, 0.0000, 0.2625, 0.2219, 0.2037, 0.3883, 0.3881, 0.2607,
        0.1453, 0.0633, 0.1122, 0.2182, 0.0832, 0.2598, 0.3783, 0.2654, 0.1461,
        0.1611, 0.0000, 0.2819, 0.2389, 

In [8]:
def standardization(matrix):
    avg = torch.mean(matrix)
    std = torch.std(matrix)
    return (matrix - avg)/std

In [9]:
lr_train2 = torch.bmm(lr_train, lr_train)
lr_stack2 = lr_train2.reshape(-1, 160)
hr_train2 = torch.bmm(hr_train, hr_train)
hr_stack2 = hr_train2.reshape(-1, 268)
print(lr_stack2.shape)

torch.Size([26720, 160])


In [10]:
lr_train2 = torch.bmm(lr_train, lr_train)
lr_stack2 = lr_train2.reshape(-1, 160)
hr_train2 = torch.bmm(hr_train, hr_train)
hr_stack2 = hr_train2.reshape(-1, 268)
print(lr_stack2.shape)

torch.Size([26720, 160])


In [11]:
def adj_hop(adj, hop):
    adj_new = adj
    for i in range(hop-1):
        adj_new = torch.bmm(adj_new, adj)
    stack_adj = adj_new.reshape(-1,adj.shape[1])
    return stack_adj 




In [12]:
print(lr_stack2)
print(adj_hop(lr_train,2))

tensor([[14.3806,  7.2528,  6.9529,  ...,  9.8382,  7.9998,  4.3059],
        [ 7.2528,  6.7310,  5.0556,  ...,  6.8077,  4.0328,  4.0265],
        [ 6.9529,  5.0556,  8.9668,  ...,  7.0718,  5.5102,  2.7241],
        ...,
        [ 5.8574,  6.6959,  5.0896,  ...,  6.7481,  2.8670,  5.2785],
        [ 1.6789,  1.6876,  2.2906,  ...,  2.8670,  7.0863,  4.0678],
        [ 4.7431,  5.3005,  4.3073,  ...,  5.2785,  4.0678,  6.6843]])
tensor([[14.3806,  7.2528,  6.9529,  ...,  9.8382,  7.9998,  4.3059],
        [ 7.2528,  6.7310,  5.0556,  ...,  6.8077,  4.0328,  4.0265],
        [ 6.9529,  5.0556,  8.9668,  ...,  7.0718,  5.5102,  2.7241],
        ...,
        [ 5.8574,  6.6959,  5.0896,  ...,  6.7481,  2.8670,  5.2785],
        [ 1.6789,  1.6876,  2.2906,  ...,  2.8670,  7.0863,  4.0678],
        [ 4.7431,  5.3005,  4.3073,  ...,  5.2785,  4.0678,  6.6843]])


In [13]:
end_dim_1 = 32

class AutoencoderLR(nn.Module):
    def __init__(self):
        super(AutoencoderLR, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(161, 128),
        nn.BatchNorm1d(128),              
        nn.ReLU(),
        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.BatchNorm1d(32),
        #nn.ReLU(),
        #nn.Linear(32, end_dim),
        )
        self.decoder = nn.Sequential(
        #nn.Linear(end_dim, 32),
        #nn.BatchNorm1d(32),
        #nn.ReLU(),
        nn.Linear(32,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(64,128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(128,160),
        )
    
    def encode(self,adj_row, hops):
        x = torch.cat([adj_row, hops], dim = 1)
        x = self.encoder(x)
        return x
    def decode(self, latent_input):
        return self.decoder(latent_input)

    def forward(self, adj_row, hoops):
        x = self.encode(adj_row, hoops)
        x = self.decode(x)
        return x


In [14]:
end_dim = 32

class AutoencoderHR(nn.Module):
    def __init__(self):
        super(AutoencoderHR, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(269, 128),
        nn.BatchNorm1d(128),              
        nn.ReLU(),
        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.BatchNorm1d(32),
        #nn.ReLU(),
        #nn.Linear(32, end_dim),
        )
        self.decoder = nn.Sequential(
        #nn.Linear(end_dim, 32),
        #nn.BatchNorm1d(32),
        #nn.ReLU(),
        nn.Linear(32,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(64,128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(128,268),
        )
    
    def encode(self,adj_row, hops):
        x = torch.cat([adj_row, hops], dim = 1)
        x = self.encoder(x)
        return x
    def decode(self, latent_input):
        return self.decoder(latent_input)

    def forward(self, adj_row, hoops):
        x = self.encode(adj_row, hoops)
        x = self.decode(x)
        return x

In [83]:
from torch.nn import functional as F

in_dim = 160

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),

            nn.Sigmoid()

        )
        self.fc_m = nn.Linear(64,32)
        self.fc_std = nn.Linear(64,32)
        self.decoder = nn.Sequential(
            
            nn.Linear(32,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            nn.Linear(64,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, in_dim)
        )
    def encode(self, x):
        h1 = self.encoder(x)
        return self.fc_m(h1), self.fc_std(h1)
    
    def reparametrize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu,logvar)
        return self.decode(z), mu, logvar
    
        

In [72]:
from torch.nn import functional as F

in_dim = 268

class VAEHR(nn.Module):
    def __init__(self):
        super(VAEHR, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),

            nn.Sigmoid()

        )
        self.fc_m = nn.Linear(64,32)
        self.fc_std = nn.Linear(64,32)
        self.decoder = nn.Sequential(
            
            nn.Linear(32,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            nn.Linear(64,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, in_dim)
        )
    def encode(self, x):
        h1 = self.encoder(x)
        return self.fc_m(h1), self.fc_std(h1)
    
    def reparametrize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu,logvar)
        return self.decode(z), mu, logvar
    
        

In [84]:


def loss_function(recon_x, x, mu, logvar):
    BCE = F.mse_loss(recon_x, x, reduction="sum")
    KLD = -0.5*torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [97]:

def train(model, adj_matrix, num_epoch = 200, lr = 0.0001, batch_size = 64):
    optimizer = torch.optim.AdamW(model.parameters(), lr = lr)
    for step in range(num_epoch):
        n_completed = 0
        while n_completed < len(adj_matrix):
            optimizer.zero_grad()
            batch = adj_matrix[n_completed: n_completed+batch_size]
            e, mu, logvar = model(batch)
            loss = loss_function(e,batch, mu, logvar)
            loss.backward()
            print(loss)
            optimizer.step()
            n_completed += batch_size
    

In [98]:
standardization(lr_stack2)

tensor([[ 0.2205, -0.2549, -0.1985,  ..., -0.0434, -0.0606, -0.4515],
        [-0.2549, -0.4769, -0.4459,  ..., -0.3602, -0.4162, -0.5998],
        [-0.1985, -0.4459, -0.3246,  ..., -0.2927, -0.3039, -0.5870],
        ...,
        [-0.4928, -0.4461, -0.4981,  ..., -0.5098, -0.6284, -0.5350],
        [-0.6933, -0.6837, -0.6068,  ..., -0.6284, -0.3828, -0.5585],
        [-0.5425, -0.5062, -0.5286,  ..., -0.5350, -0.5585, -0.5276]])

In [99]:
print(lr_stack2)
norm = standardization(lr_stack2)
print(norm)

tensor([[335.2887, 189.7507, 206.9984,  ..., 254.4923, 249.2381, 129.5542],
        [189.7507, 121.7569, 131.2713,  ..., 157.4972, 140.3612,  84.1543],
        [206.9984, 131.2713, 168.4046,  ..., 178.1749, 174.7270,  88.0600],
        ...,
        [116.8940, 131.2039, 115.2872,  ..., 111.7079,  75.3803, 103.9770],
        [ 55.5164,  58.4502,  81.9970,  ...,  75.3803, 150.5891,  96.7700],
        [101.6696, 112.8067, 105.9391,  ..., 103.9770,  96.7700, 106.2328]])
tensor([[ 0.2205, -0.2549, -0.1985,  ..., -0.0434, -0.0606, -0.4515],
        [-0.2549, -0.4769, -0.4459,  ..., -0.3602, -0.4162, -0.5998],
        [-0.1985, -0.4459, -0.3246,  ..., -0.2927, -0.3039, -0.5870],
        ...,
        [-0.4928, -0.4461, -0.4981,  ..., -0.5098, -0.6284, -0.5350],
        [-0.6933, -0.6837, -0.6068,  ..., -0.6284, -0.3828, -0.5585],
        [-0.5425, -0.5062, -0.5286,  ..., -0.5350, -0.5585, -0.5276]])


In [100]:
lr_test_stack = lr_test.reshape(-1,160)
print(lr_test.shape)
lr_test_stack2 = adj_hop(lr_test,3)


torch.Size([112, 160, 160])


In [101]:
model = VAE()
params = sum(p.numel() for p in model.parameters() if p.requires_grad_)
print(params)
hr_stack2 = adj_hop(hr_train,3)
norm = standardization(lr_test_stack2)

train(model, norm)


64864
tensor(4162.5210, grad_fn=<AddBackward0>)
tensor(4514.8433, grad_fn=<AddBackward0>)
tensor(4256.1577, grad_fn=<AddBackward0>)
tensor(3429.2229, grad_fn=<AddBackward0>)
tensor(3343.1106, grad_fn=<AddBackward0>)
tensor(3589.0869, grad_fn=<AddBackward0>)
tensor(4710.3301, grad_fn=<AddBackward0>)
tensor(3672.2773, grad_fn=<AddBackward0>)
tensor(3096.4580, grad_fn=<AddBackward0>)
tensor(3400.7278, grad_fn=<AddBackward0>)
tensor(4491.1440, grad_fn=<AddBackward0>)
tensor(5364.9917, grad_fn=<AddBackward0>)
tensor(5779.4258, grad_fn=<AddBackward0>)
tensor(6109.8770, grad_fn=<AddBackward0>)
tensor(5339.8247, grad_fn=<AddBackward0>)
tensor(5220.3965, grad_fn=<AddBackward0>)
tensor(20949.4844, grad_fn=<AddBackward0>)
tensor(9458.9707, grad_fn=<AddBackward0>)
tensor(3869.1311, grad_fn=<AddBackward0>)
tensor(4815.0713, grad_fn=<AddBackward0>)
tensor(4632.5889, grad_fn=<AddBackward0>)
tensor(5303.5854, grad_fn=<AddBackward0>)
tensor(3802.2859, grad_fn=<AddBackward0>)
tensor(3966.0237, grad_fn=<

In [102]:
model.eval()
mu, logvar = model.encode(norm)
res_lr_1 = model.reparametrize(mu,logvar).reshape(112, 160, 32)
torch.save(res_lr_1.detach(), "model_autoencoder/final_embeddings/encode_lr_test_3.pt")


In [ ]:
torch.save(model.state_dict(), "model_autoencoder/autoencoder.pth")